ENSEMBLES

In [1]:
%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

Decision tress

In [2]:
from sklearn.model_selection import train_test_split

X = pd.read_csv('X_opening.csv')
y = X['worldwide_gross']
X = X.drop('worldwide_gross',axis=1)

In [3]:
X_train, X_test, y_train, y_test = train_test_split(X,y,random_state=1)

In [4]:
from sklearn.tree import DecisionTreeRegressor

model = DecisionTreeRegressor(max_depth=2)

In [5]:
model.fit(X_train,y_train)

DecisionTreeRegressor(max_depth=2)

In [7]:
import graphviz 

In [10]:
from sklearn.tree import export_graphviz

treedot = export_graphviz(model, out_file=None, feature_names = X.columns)

In [11]:
treedot

'digraph Tree {\nnode [shape=box] ;\n0 [label="opening_gross <= 41613376.0\\nmse = 4.491994363696598e+16\\nsamples = 1665\\nvalue = 141540319.054"] ;\n1 [label="opening_gross <= 22074047.0\\nmse = 1.333382219312781e+16\\nsamples = 1506\\nvalue = 92999937.199"] ;\n0 -> 1 [labeldistance=2.5, labelangle=45, headlabel="True"] ;\n2 [label="mse = 4923666241203554.0\\nsamples = 1257\\nvalue = 64781848.271"] ;\n1 -> 2 ;\n3 [label="mse = 3.147813101998751e+16\\nsamples = 249\\nvalue = 235450289.735"] ;\n1 -> 3 ;\n4 [label="opening_gross <= 70351576.0\\nmse = 1.1039811871593606e+17\\nsamples = 159\\nvalue = 601300162.289"] ;\n0 -> 4 [labeldistance=2.5, labelangle=-45, headlabel="False"] ;\n5 [label="mse = 4.067538845924512e+16\\nsamples = 92\\nvalue = 440868287.554"] ;\n4 -> 5 ;\n6 [label="mse = 1.2226485798747622e+17\\nsamples = 67\\nvalue = 821594676.851"] ;\n4 -> 6 ;\n}'

In [15]:
#graphviz.Source(treedot)

RANDOM FOREST

In [22]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import cross_validate

forest  = RandomForestRegressor(200)
results = cross_validate(forest,X,y,cv=5,return_train_score=True,scoring='r2')

In [23]:
train_scores = results['train_score']
test_scores = results['test_score']
print(np.mean(train_scores))
print(np.mean(test_scores))

#HAY OVERFITTING!
#Mejor resultado que Lasso! Ya no hay underfitting y hay un mejor score r2. Sin embargo hay una diferencia importante entre score de entrenamiento y de test(overfit)

0.965249492366542
0.517657530974039


GRADIENT BOOSTED TREES

In [24]:
from sklearn.ensemble import GradientBoostingRegressor

ensemble  = GradientBoostingRegressor()
results = cross_validate(ensemble,X,y,cv=5,return_train_score=True,scoring='r2')

In [25]:
train_scores = results['train_score']
test_scores = results['test_score']
print(np.mean(train_scores))
print(np.mean(test_scores))

#Exactamente lo mismo que Random Forest

0.9151392143549633
0.52431822813825


OPTIMIZACION DE PARAMETROS

In [26]:
from sklearn.model_selection import train_test_split

X_train,X_test,y_train,y_test = train_test_split(X,y,random_state=1)

In [27]:
from sklearn.model_selection import GridSearchCV

#Definir combinacion de parametros
param_test1 = {'n_estimators':range(20,501,20)}

In [28]:
list(param_test1['n_estimators'])

[20,
 40,
 60,
 80,
 100,
 120,
 140,
 160,
 180,
 200,
 220,
 240,
 260,
 280,
 300,
 320,
 340,
 360,
 380,
 400,
 420,
 440,
 460,
 480,
 500]

In [29]:
estimator = GradientBoostingRegressor(learning_rate=0.1,min_samples_split=500,min_samples_leaf=50,max_depth=8,
max_features='sqrt',subsample=0.8,random_state=10)

In [30]:
gsearch = GridSearchCV(estimator,param_grid=param_test1,scoring='r2',cv=5)

In [31]:
gsearch.fit(X_train,y_train)

GridSearchCV(cv=5,
             estimator=GradientBoostingRegressor(max_depth=8,
                                                 max_features='sqrt',
                                                 min_samples_leaf=50,
                                                 min_samples_split=500,
                                                 random_state=10,
                                                 subsample=0.8),
             param_grid={'n_estimators': range(20, 501, 20)}, scoring='r2')

In [36]:
gsearch.cv_results_,gsearch.best_params_,gsearch.best_score_

({'mean_fit_time': array([0.20597792, 0.10754271, 0.17056165, 0.21617689, 0.28824406,
         0.32134128, 0.3713387 , 0.42474566, 0.47700005, 0.54094911,
         0.57598987, 0.64317575, 0.65987515, 0.73912573, 0.77331138,
         0.88867149, 0.91414232, 0.94936447, 1.01085687, 1.10976496,
         1.14001923, 1.21151805, 1.27629609, 1.26240191, 1.34390602]),
  'std_fit_time': array([0.30845447, 0.00351872, 0.00873967, 0.00515952, 0.01146196,
         0.00808841, 0.00668596, 0.0128506 , 0.01412208, 0.04243156,
         0.01882462, 0.03453891, 0.01789663, 0.03932329, 0.02450427,
         0.05467872, 0.02374361, 0.0141751 , 0.02698495, 0.03089718,
         0.0204956 , 0.03124998, 0.04712557, 0.04208516, 0.07167714]),
  'mean_score_time': array([0.00690618, 0.00587416, 0.00561786, 0.00600681, 0.00479913,
         0.00790091, 0.00664968, 0.00939016, 0.00815196, 0.00753527,
         0.00799503, 0.00878243, 0.00858469, 0.00835524, 0.00916748,
         0.01071258, 0.01046581, 0.01060562, 0.

In [41]:
final_results = cross_validate(gsearch.best_estimator_,X_train,y_train,return_train_score=True)

In [42]:
train_scores = final_results['train_score']
test_scores = final_results['test_score']
print(np.mean(train_scores))
print(np.mean(test_scores))

#Nos encontramos en una relacion ideal

0.8286608694466742
0.7553059694284988


In [43]:
estimator = GradientBoostingRegressor(learning_rate=0.1,min_samples_split=500,min_samples_leaf=50,max_depth=8,
max_features='sqrt',subsample=0.8,random_state=10,n_estimators=240)

In [44]:
estimator.fit(X_train,y_train)

GradientBoostingRegressor(max_depth=8, max_features='sqrt', min_samples_leaf=50,
                          min_samples_split=500, n_estimators=240,
                          random_state=10, subsample=0.8)

In [45]:
estimator.score(X_test,y_test)
#Excelente score! 

0.8092888852563106